# Comparison
In this script we demonstrate the use of the comparison function implemented in CICADA
There are a couple choices available which change the behavior of the script

if random_ops is true, testVal1 and 2 will be ignored and randomly generated values will be used <br>
if random_ops is false, the values that will be compared are set by testVal1 and 2 so you can pick your operands
numRuns is really only useful if you want to confirm correctness for several randomized tests in a series, feel free to set it to 1 if you are interested to compare testVal1 and 2, or leave it more than one to see that the algorithm is stable in returning correct expected results. It is not a probabilistic comparison.

When everything is finished we print some summary statistics and performance information. The statistics module will complain if you set numRuns to 1.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator
from tqdm import *
from time import time
from statistics import mean, stdev
logging.basicConfig(level=logging.INFO)

In [4]:
random_ops = True
testVal1 = numpy.array(0)
testVal2 = numpy.array(-8)
numRuns = 5
#########################################

In [5]:
results = []
errors = {}
times = []
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    for i in tqdm(range(numRuns)):
        if random_ops:
            bit_share1, secret_share1 = protocol.random_bitwise_secret(generator=generator, bits=64)
            bit_share2, secret_share2 = protocol.random_bitwise_secret(generator=generator, bits=64)
        else:
            secret_share1 = protocol.share(src=0, secret=protocol.encoder.encode(testVal1), shape=testVal1.shape)
            secret_share2 = protocol.share(src=0, secret=protocol.encoder.encode(testVal2), shape=testVal2.shape)
        secret1 = protocol.encoder.decode(protocol.reveal(secret_share1))
        secret2 = protocol.encoder.decode(protocol.reveal(secret_share2))
        t0 = time()
        lt = protocol.less(lhs=secret_share1, rhs=secret_share2)
        times.append(time()-t0)
        revealed_lt = protocol.reveal(lt)
        if revealed_lt == 1 and secret1 < secret2:
            results.append(True)
        elif revealed_lt == 0 and secret1 >= secret2:
            results.append(True)
        else:
            results.append(False)
            errors[i]=(secret1, secret2, revealed_lt)
    log.info(f"Player {communicator.rank} {secret1} {'<' if revealed_lt else '>='} {secret2}")
    if not all(results):
        print(f'Num errors: {sum([1 for x in results if x==False])}')
        for k, v in errors.items():
            if v[2] == 1:
                symbol = '<'
            elif v[2] == 0:
                symbol = '>='
            else:
                symbol = '?'
            print(f'Run # {k}\n\t{v[0]} {symbol} {v[1]}')
    else: 
        print('No Errors!')
    print(f'Avg time: {mean(times)}\nStdev time: {stdev(times)}')


In [6]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:49527 from tcp://127.0.0.1:49527.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:49527 from tcp://127.0.0.1:49528.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:49527 from tcp://127.0.0.1:49529.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.25s/it]

INFO:root:Player 0 -83261057422014.44 < 135767166994149.25
INFO:root:Player 1 -83261057422014.44 < 135767166994149.25
INFO:root:Player 2 -83261057422014.44 < 135767166994149.25


No Errors!No Errors!No Errors!

Avg time: 3.165136384963989
Stdev time: 0.0584495077475798

Avg time: 3.1538023471832277
Stdev time: 0.06014930208389712Avg time: 3.157222938537598
Stdev time: 0.059930800927650504


INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.


INFO:cicada.communicator.nng:Comm 'world' player 2 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None
INFO:cicada.communicator.nng:Player 2 returned: None


[None, None, None]